In [2]:
import pymorphy2
import nltk
from os import path

import context as c

In [3]:
book_name = path.join(c.DATA_PATH, 'капитал.txt')

In [4]:
f = open(book_name)
raw = f.read()
num_char = len(raw)

In [5]:
class PyMorphyTagger():
    def __init__(self) -> None:
        self.morph = pymorphy2.MorphAnalyzer()

    def tag(self, word):
        return self.morph.parse(word)[0].tag.POS

# Тэггер, который никогда не даёт однозначного ответа
class CustomTagger():
    def tag(self, word):
        # Источник - рандомные школьные презентации в интернете
        adj_end = ['ой', 'ий', 'ый', 'ого', 'его', 'ому', 'ему', 'им', 'ым', 'ом', 'ем', 'ое', 'ее', 'ая', 'яя', 'ей', 'ую', 'юю', 'ие', 'ые', 'их', 'ых', 'ими', 'ыми']
        adv_beg = ['на', 'за', 'в', 'из', 'ис', 'до', 'с']
        adv_end = ['о', 'а']
        vrb_end = ['у', 'ю', 'ем', 'ешь', 'ете', 'ет', 'ют', 'ут', 'у', 'ю', 'им', 'ишь', 'ите', 'ит', 'ят', 'ат']
        vrb_inf_end = ['ть', 'ться', 'тся']

        res = []
        if CustomTagger.check_end(word, adv_end) and CustomTagger.check_beg(word, adv_beg):
            res.append('ADV')

        if CustomTagger.check_end(word, adj_end):
            res.append('ADJ')
       
        if CustomTagger.check_end(word, vrb_end) or CustomTagger.check_end(word, vrb_inf_end):
            res.append('VERB')
        
        # если слово не наречье и не глагол в инфинитиве, возможно, оно существительное
        if ('ADV' not in res) and not CustomTagger.check_end(word, vrb_inf_end):
            res.append('NOUN')

        return res
    
    def check_end(word, endings):
        for end in endings:
            if word.endswith(end):
                return True
        return False
    
    def check_beg(word, endings):
        for end in endings:
            if word.startswith(end):
                return True
        return False
        


In [6]:
tokenizer = nltk.NLTKWordTokenizer()
morph = pymorphy2.MorphAnalyzer()

In [7]:
def print_text_stats(words, unique_words, name, stopwords = set()):
    # пробуем очистить от стопслов без предварительной нормализации
    words_count = len(words)
    unique_words_clean = unique_words - stopwords

    # пробуем очистить от стопслов после нормализации
    unique_words_norm = {morph.parse(w)[0].normal_form for w in unique_words}
    unique_words_norm_clean = unique_words_norm - stopwords

    unique_words_count = len(unique_words_clean)
    unique_words_norm_count = len(unique_words_norm_clean)

    print(f'Статистика по файлу {name}')
    print('    Количество слов в тексте:', words_count)
    print('    Количество уникальных слов:', unique_words_count)
    print('    Количество уникальных слов в нормальной форме:', unique_words_norm_count)
    print('    Лексическое разнообразие:', unique_words_count / words_count)
    print('    Лексическое разнообразие в нормальной форме:', unique_words_norm_count / words_count)

In [8]:
words = tokenizer.tokenize(raw)
words = [w.lower() for w in words]
words = [w for w in words if w.isalpha()]

unique_words = set(words)
stopwords = set(nltk.corpus.stopwords.words('russian'))

print_text_stats(words, unique_words, book_name)
print_text_stats(words, unique_words, 'чистый_' + book_name, stopwords)

Статистика по файлу data\капитал.txt
    Количество слов в тексте: 360993
    Количество уникальных слов: 27972
    Количество уникальных слов в нормальной форме: 11911
    Лексическое разнообразие: 0.07748626704672944
    Лексическое разнообразие в нормальной форме: 0.03299509962797063
Статистика по файлу чистый_data\капитал.txt
    Количество слов в тексте: 360993
    Количество уникальных слов: 27823
    Количество уникальных слов в нормальной форме: 11806
    Лексическое разнообразие: 0.07707351666098788
    Лексическое разнообразие в нормальной форме: 0.032704235262179596


In [9]:
taggers = [PyMorphyTagger(), CustomTagger()]
selected_words = list(unique_words)[1010:1018]
print('Выбраны слова для проверки: ', selected_words)

for tagger in taggers:
    print(f'Проверяем {tagger.__class__.__name__}:')
    for w in selected_words:
        print('   ', w, tagger.tag(w))

Выбраны слова для проверки:  ['уравновешенным', 'долями', 'трудностей', 'создание', 'привел', 'достаточный', 'стоила', 'машина']
Проверяем PyMorphyTagger:
    уравновешенным PRTF
    долями NOUN
    трудностей NOUN
    создание NOUN
    привел VERB
    достаточный ADJF
    стоила VERB
    машина NOUN
Проверяем CustomTagger:
    уравновешенным ['ADJ', 'NOUN']
    долями ['NOUN']
    трудностей ['ADJ', 'NOUN']
    создание ['ADJ', 'NOUN']
    привел ['NOUN']
    достаточный ['ADJ', 'NOUN']
    стоила ['ADV']
    машина ['NOUN']
